# Lightweight Fine-Tuning Project


* PEFT technique: Lora
* Model: gpt2 with a classifier added
* Evaluation approach: The Trainer evaluate method is used with accuracy as metric. 
    * First the model with untrained classifier is evaluated
    * Next the classifier is trained and evaluated
    * Next the entire model is fine tuned using Peft Lora
    * Finally, the fined tuned model is evaluted
* Fine-tuning dataset: rikka-snow/prompt-injection-multilingual

## Loading and Evaluating a Foundation Model

### Load a pretrained HF model

In [1]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "gpt2",
    num_labels=2,
    id2label={0: "Normal", 1: "Prompt Injection"},  # For converting predictions to strings
    label2id={"Normal": 0, "Prompt Injection": 1},
)
model.config.pad_token_id = model.config.eos_token_id

# Freeze all the parameters of the base model
for param in model.base_model.parameters():
    param.requires_grad = False

# Show the model architecture
print(model)


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=2, bias=False)
)


### Load and preprocess a dataset

In [2]:
from datasets import load_dataset
# Load the train and test splits of the imdb dataset
splits = ["train", "test"]

ds = {split: ds for split, ds in zip(splits, load_dataset("rikka-snow/prompt-injection-multilingual", split=splits))}

# Thin out the dataset to make it run faster for this example
for split in splits:
    ds[split] = ds[split].shuffle(seed=42).select(range(500))

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

def preprocess_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_ds = {}
for split in splits:
    tokenized_ds[split] = ds[split].map(preprocess_function, batched=True)



Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

### Evaluate the pretrained model


In [4]:
import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./data/prompt_classification1",
        learning_rate=2e-3,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=2,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer, padding="max_length"),
    compute_metrics=compute_metrics,
)

trainer.evaluate()

c:\Users\z00110mk\git\Apply-Lightweight-Fine-Tuning-to-a-Foundation-Model\.venv\lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\z00110mk\AppData\Local\Temp\ipykernel_15452\2592870431.py:10: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


  0%|          | 0/32 [00:00<?, ?it/s]

{'eval_loss': 4.234659194946289,
 'eval_model_preparation_time': 0.0016,
 'eval_accuracy': 0.516,
 'eval_runtime': 469.1167,
 'eval_samples_per_second': 1.066,
 'eval_steps_per_second': 0.068}

As the classifier was initialized without training, the accuracy is bad (around 0.5). Next we train the classifier only before fine tuning the entire model. 

In [5]:
trainer.train()

  0%|          | 0/64 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

{'eval_loss': 0.6386556625366211, 'eval_model_preparation_time': 0.0016, 'eval_accuracy': 0.612, 'eval_runtime': 484.3469, 'eval_samples_per_second': 1.032, 'eval_steps_per_second': 0.066, 'epoch': 1.0}


  0%|          | 0/32 [00:00<?, ?it/s]

{'eval_loss': 0.554269552230835, 'eval_model_preparation_time': 0.0016, 'eval_accuracy': 0.71, 'eval_runtime': 470.5684, 'eval_samples_per_second': 1.063, 'eval_steps_per_second': 0.068, 'epoch': 2.0}
{'train_runtime': 2670.8686, 'train_samples_per_second': 0.374, 'train_steps_per_second': 0.024, 'train_loss': 0.8240573406219482, 'epoch': 2.0}


TrainOutput(global_step=64, training_loss=0.8240573406219482, metrics={'train_runtime': 2670.8686, 'train_samples_per_second': 0.374, 'train_steps_per_second': 0.024, 'total_flos': 522593501184000.0, 'train_loss': 0.8240573406219482, 'epoch': 2.0})

The classifier is now trained and eval_accuracy should be around 0.7. The accuracy has improved as expected.

## Performing Parameter-Efficient Fine-Tuning


### Create a PEFT model

In [6]:
from peft import LoraConfig, get_peft_model, TaskType
import torch

# freeze all the weights
for param in model.parameters():
    param.requires_grad = False

# LoRa
config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_CLS
)

peft_model = get_peft_model(model, config)

peft_model.print_trainable_parameters()

trainable params: 591,360 || all params: 125,032,704 || trainable%: 0.4730


c:\Users\z00110mk\git\Apply-Lightweight-Fine-Tuning-to-a-Foundation-Model\.venv\lib\site-packages\peft\tuners\lora\layer.py:1150: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


### Train the PEFT model

In [ ]:
trainer = Trainer(
    model=peft_model,
    args=TrainingArguments(
        output_dir="./data/prompt_classification2",
        learning_rate=2e-3,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        num_train_epochs=2,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer, padding="max_length"),
    compute_metrics=compute_metrics,
)

trainer.train()



c:\Users\z00110mk\git\Apply-Lightweight-Fine-Tuning-to-a-Foundation-Model\.venv\lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\z00110mk\AppData\Local\Temp\ipykernel_15452\410219606.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


  0%|          | 0/250 [00:00<?, ?it/s]

Now the we should be at around 0.8 😃

### Save the PEFT model

In [ ]:
merged_model = peft_model.merge_and_unload()
merged_model.save_pretrained(".", safe_serialization=True)

## Performing Inference with a PEFT Model

### Load the saved PEFT model

In [ ]:
loaded_model = AutoModelForSequenceClassification.from_pretrained(".")

### Evaluate the fine-tuned model

In [ ]:
trainer = Trainer(
    model=loaded_model,
    args=TrainingArguments(
        output_dir="./data/prompt_classification3",
        evaluation_strategy="epoch",
    ),
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer, padding="max_length"),
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.evaluate()

{'eval_loss': 0.3747003674507141,
 'eval_accuracy': 0.816,
 'eval_runtime': 43.3697,
 'eval_samples_per_second': 11.529,
 'eval_steps_per_second': 1.453}